In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import random
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader,IterableDataset

In [ ]:
def clean_ct_data(oppScrData):
    # Delete rows with empty values
    ct_data= oppScrData[["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'Age at CT']]
    n = ct_data.shape[0]
    preprocessed_ct_data = []
    for i in range(n):
        row = ct_data.loc[i]
        ignore = False
        for j in row:
          if pd.isna(j):
            ignore = True
            break
        if not ignore:
          preprocessed_ct_data.append(row)
    return np.array(preprocessed_ct_data)

oppScrData = pd.read_excel (r'sample_data/OppScrData.xlsx')  
d = clean_ct_data(oppScrData)
print(len(d))

In [2]:
class CT_Dataset(Dataset):

    def __init__(self):
        self.oppScrData = pd.read_excel (r'sample_data/OppScrData.xlsx')        

    def __len__(self):
        return len(self.oppScrData)

    def __getitem__(self, idx):
        a = self.oppScrData.iloc[idx]
        ct_data = a[["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)']]
        age = a[['Age at CT']]
        ct_data = np.array(ct_data, dtype=np.float32)       
        age = np.array(age, dtype=np.float32)       
        return torch.from_numpy(ct_data), torch.from_numpy(age)

In [3]:
a = CT_Dataset()

FileNotFoundError: ignored

In [ ]:
# b = DataLoader(a, batch_size=32, shuffle=True)
b = DataLoader(a)

In [ ]:
def build_model():
  return nn.Sequential(
    nn.Linear(11, 32),
    nn.ReLU(),
    nn.Linear(32, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
    )

model = build_model()

In [ ]:
criterion = nn.MSELoss()

def train_model(model, train_loader, criterion, T):
  model.train()
  opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  for epoch in range(T): 
      for data in enumerate(train_loader):
          ct_data, actual_age = data[1][0], data[1][1]
          opt.zero_grad()
          predicted_age = model(ct_data)          
          loss = criterion(predicted_age, actual_age)
          loss.backward() 
          opt.step()
      
      print("​Train Epoch: "+str(epoch))

  model.train(mode=False)

In [ ]:
train_model(model, b, criterion, T = 1)